- for ngc 2998 if the picture given looks weird it's probably the spectrum pic
- darks are the same length as images you want to reduce (but you can also add some together)
- create a table that lists file names, filter, exposure time, and kind of file
- manually check for correct settings by downloading and opening fits file (don't trust preview screen)
- keep original data as raw
- [ccd data reduction guide](https://ccdproc.readthedocs.io/en/latest/)